In [1]:
import os
import keras_hub
import keras

import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

2025-11-13 08:34:18.558051: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763019258.569368   10562 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763019258.572842   10562 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763019258.582762   10562 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763019258.582771   10562 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763019258.582772   10562 computation_placer.cc:177] computation placer alr

In [2]:
# Data
BATCH_SIZE = 64
MIN_STRING_LEN = 512  # Strings shorter than this will be discarded
SEQ_LEN = 128  # Length of training sequences, in tokens

# Model
EMBED_DIM = 256
FEED_FORWARD_DIM = 128
NUM_HEADS = 3
NUM_LAYERS = 2
VOCAB_SIZE = 5000  # Limits parameters in model.

# Training
EPOCHS = 5

# Inference
NUM_TOKENS_TO_GENERATE = 80

In [7]:
#keras.utils.get_file(
    #origin="https://dldata-public.s3.us-east-2.amazonaws.com/simplebooks.zip",
    #extract=True,
#)
dir = os.path.expanduser("~/.keras/datasets/simplebooks/simplebooks/")

# Load simplebooks-92 train set and filter out short lines.
raw_train_ds = (
    tf_data.TextLineDataset(dir + "simplebooks-92-raw/train.txt")
    .filter(lambda x: tf_strings.length(x) > MIN_STRING_LEN)
    .batch(BATCH_SIZE)
    .shuffle(buffer_size=256)
)

# Load simplebooks-92 validation set and filter out short lines.
raw_val_ds = (
    tf_data.TextLineDataset(dir + "simplebooks-92-raw/valid.txt")
    .filter(lambda x: tf_strings.length(x) > MIN_STRING_LEN)
    .batch(BATCH_SIZE)
)

In [8]:
# Train tokenizer vocabulary
vocab = keras_hub.tokenizers.compute_word_piece_vocabulary(
    raw_train_ds,
    vocabulary_size=VOCAB_SIZE,
    lowercase=True,
    reserved_tokens=["[PAD]", "[UNK]", "[BOS]"],
)

In [9]:
tokenizer = keras_hub.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    sequence_length=SEQ_LEN,
    lowercase=True,
)

In [10]:
# packer adds a start token
start_packer = keras_hub.layers.StartEndPacker(
    sequence_length=SEQ_LEN,
    start_value=tokenizer.token_to_id("[BOS]"),
)


def preprocess(inputs):
    outputs = tokenizer(inputs)
    features = start_packer(outputs)
    labels = outputs
    return features, labels


# Tokenize and split into train and label sequences.
train_ds = raw_train_ds.map(preprocess, num_parallel_calls=tf_data.AUTOTUNE).prefetch(
    tf_data.AUTOTUNE
)
val_ds = raw_val_ds.map(preprocess, num_parallel_calls=tf_data.AUTOTUNE).prefetch(
    tf_data.AUTOTUNE
)

In [11]:
inputs = keras.layers.Input(shape=(None,), dtype="int32")
# Embedding.
embedding_layer = keras_hub.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=SEQ_LEN,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)
x = embedding_layer(inputs)
# Transformer decoders.
for _ in range(NUM_LAYERS):
    decoder_layer = keras_hub.layers.TransformerDecoder(
        num_heads=NUM_HEADS,
        intermediate_dim=FEED_FORWARD_DIM,
    )
    x = decoder_layer(x)  # Giving one argument only skips cross-attention.
# Output.
outputs = keras.layers.Dense(VOCAB_SIZE)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
perplexity = keras_hub.metrics.Perplexity(from_logits=True, mask_token_id=0)
model.compile(optimizer="adam", loss=loss_fn, metrics=[perplexity])

In [12]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, None, 256)      │     1,312,768 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder             │ (None, None, 256)      │       329,085 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_1           │ (None, None, 256)      │       329,085 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 5000)     │     1,285,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,255,938 (12.42 MB)

 Trainable params: 3,255,938 (12.42 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

Epoch 1/5


/home/xy/Desktop/ml/GDL/tf_2.19/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'position_embedding' (of type PositionEmbedding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/xy/Desktop/ml/GDL/tf_2.19/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/xy/Desktop/ml/GDL/tf_2.19/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers 

     12/Unknown 9s 16ms/step - loss: 7.9659 - perplexity: 3073.0454

I0000 00:00:1763019674.073953   10723 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   2390/Unknown 64s 23ms/step - loss: 5.0002 - perplexity: 182.0752

...

2025-11-13 08:45:42.538697: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2445/2445 ━━━━━━━━━━━━━━━━━━━━ 70s 27ms/step - loss: 3.9492 - perplexity: 51.9189 - val_loss: 3.9813 - val_perplexity: 53.6526
Epoch 5/5


2025-11-13 08:45:42.757186: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5942467455585689329
2025-11-13 08:45:42.757201: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 377852895275011384


2445/2445 ━━━━━━━━━━━━━━━━━━━━ 71s 28ms/step - loss: 3.9056 - perplexity: 49.7042 - val_loss: 3.9618 - val_perplexity: 52.6263


2025-11-13 08:46:54.092631: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5942467455585689329
2025-11-13 08:46:54.092648: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 377852895275011384


In [14]:
# The "packer" layers adds the [BOS] token for us.
prompt_tokens = start_packer(tokenizer([""]))
prompt_tokens

<tf.Tensor: shape=(1, 128), dtype=int32, numpy=
array([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>

In [15]:
def next(prompt, cache, index):
    logits = model(prompt)[:, index - 1, :]
    # Ignore hidden states for now; only needed for contrastive search.
    hidden_states = None
    return logits, hidden_states, cache

In [16]:
sampler = keras_hub.samplers.GreedySampler()
output_tokens = sampler(
    next=next,
    prompt=prompt_tokens,
    index=1,  # Start sampling immediately after the [BOS] token.
)
txt = tokenizer.detokenize(output_tokens)
print(f"Greedy search generated text: \n{txt}\n")

Greedy search generated text: 
['[BOS] the next morning the sun was setting , and the sun was shining in the sky , and the sun was shining brightly , and the sun shone brightly , and the sun shone brightly in the sky . the sun was shining bright and bright , and the stars shone bright and brighter and brighter and brighter and brighter , and the stars shone brighter and brighter and brighter and brighter and brighter and brighter and brighter , and the sun shone brighter and brighter and brighter and brighter and brighter , and the sun shone brighter and brighter and brighter sky . the']



In [17]:
sampler = keras_hub.samplers.BeamSampler(num_beams=10)
output_tokens = sampler(
    next=next,
    prompt=prompt_tokens,
    index=1,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Beam search generated text: \n{txt}\n")

Beam search generated text: 
['[BOS] " i don \' t believe it , " he said . " i don \' t believe it , but i don \' t believe it . i don \' t believe it , but i don \' t believe it . i don \' t believe it , and i don \' t believe it , but i don \' t believe it . i don \' t believe it , but i don \' t believe it . i don \' t believe it , but i don \' t believe it . i don \' t believe it , but i don \' t believe it . i don \' t believe it , but i don \' t think it \' s']



In [18]:
sampler = keras_hub.samplers.RandomSampler()
output_tokens = sampler(
    next=next,
    prompt=prompt_tokens,
    index=1,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Random search generated text: \n{txt}\n")

Random search generated text: 
['[BOS] then the bear roared , although the noise was coming down , frank felt obliged to hear and his jest journey . he leaped from his pack assag down on a transmitter of summer . he was angry , indeed , but this trail was apparently swayed off as his journey . henry , their bear fell with danes quickly , and their first shot came , so that , that the blood was cool between them and he still rolled along beside his muttours before hemmed . what time he at last , and when dark or tired , who had come over till he reached the spot where he']



In [19]:
sampler = keras_hub.samplers.TopKSampler(k=10)
output_tokens = sampler(
    next=next,
    prompt=prompt_tokens,
    index=1,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-K search generated text: \n{txt}\n")

Top-K search generated text: 
['[BOS] they found the two trampings at the same pace , which seemed , to be the case , to see which in the clump . they were not accustomed to do so , and they were in motion they could not go . they looked at this , though they did not see the difference between the eyes . it was so strange that they had no chance of being affected . but the two lads did not understand that they did not know what their own way was passing . they were very tired and they were not quite so far as they did . they had to look at their heads like a white , and the little']



In [20]:
sampler = keras_hub.samplers.TopPSampler(p=0.5)
output_tokens = sampler(
    next=next,
    prompt=prompt_tokens,
    index=1,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-P search generated text: \n{txt}\n")

Top-P search generated text: 
['[BOS] it was now late in the afternoon , and , on the evening , there was no prospect of crossing the river , and the forest , they would go in the river , and then they began to cry . in a few moments they had seen that the water had been in the water , and had already gathered in the valley , and the water scatters had made a dash for them . it was in the morning that the stream would fall into the water , and they would have to go up the river and the other shore . the night was dark , and the sun was hot . the sun was rising high , and the water']



In [21]:
class TopKTextGenerator(keras.callbacks.Callback):
    """A callback to generate text from a trained model using top-k."""

    def __init__(self, k):
        self.sampler = keras_hub.samplers.TopKSampler(k)

    def on_epoch_end(self, epoch, logs=None):
        output_tokens = self.sampler(
            next=next,
            prompt=prompt_tokens,
            index=1,
        )
        txt = tokenizer.detokenize(output_tokens)
        print(f"Top-K search generated text: \n{txt}\n")


text_generation_callback = TopKTextGenerator(k=10)
# Dummy training loop to demonstrate callback.
model.fit(train_ds.take(1), verbose=2, epochs=2, callbacks=[text_generation_callback])

Epoch 1/2


2025-11-13 08:47:15.491495: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5942467455585689329
2025-11-13 08:47:15.491515: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 377852895275011384


Top-K search generated text: 
['[BOS] the boys were at a little group who had been sitting in bed at the head of a little table and a little tableau ; but the two girls were not quite as much as the same . they were all very busy with their own pluming and accomots , and were all very well acquainted with the various prefects , and were not the same as the primitive , as one after another of the others , but it was not possible that they would be a great deal more than a match . they were now at all , for , and there were many of the girls , who']

1/1 - 7s - 7s/step - loss: 3.8012 - perplexity: 44.7686
Epoch 2/2


2025-11-13 08:47:22.151385: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5942467455585689329
2025-11-13 08:47:22.151409: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 377852895275011384


Top-K search generated text: 
["[BOS] the screen was now getting up . the saxons ' discussion were to do so , and the conway was not slow , that he was going back in his seat . but , as he had , the time when he got a little better , he was sure , he was very sorry that he was able to keep the boy on his feet and he went to the door of the house and his mother , and then began to talk about him . he was in a very small room , and he had to be a great many times more than before , so he could hardly see whether she could"]

1/1 - 7s - 7s/step - loss: 4.1391 - perplexity: 62.7775
